In [86]:
import sqlite3
import pandas as pd
import math
from scipy import stats
import seaborn as sns
import numpy as np

In [87]:
connection = sqlite3.connect('Northwind_small.sqlite')
cursor = connection.cursor()

In [88]:
def sql_select_to_df(SQL_COMMAND, cursor=cursor):
    results = cursor.execute(SQL_COMMAND).fetchall()
    df = pd.DataFrame(results)
    df.columns = [i[0] for i in cursor.description]
    return df

In [96]:
orders_cities = sql_select_to_df("""SELECT *
                                FROM [Customer] AS C
                                JOIN [Order] AS O
                                ON O.CustomerID = C.Id
                        """)


In [97]:
pd.set_option('display.max_columns', 500)
orders_cities

,Id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,VINET,Vins et alcools Chevalier,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,TOMSP,Toms Spezialitäten,Karin Josephs,Marketing Manager,Luisenstr. 48,Münster,Western Europe,44087,Germany,0251-031259,0251-035695,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,HANAR,Hanari Carnes,Mario Pontes,Accounting Manager,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil,(21) 555-0091,(21) 555-8765,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,VICTE,Victuailles en stock,Mary Saveley,Sales Agent,"2, rue du Commerce",Lyon,Western Europe,69004,France,78.32.54.86,78.32.54.87,10251,VICTE,3,2012-07-08,2012-08-05,2012-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,SUPRD,Suprêmes délices,Pascale Cartrain,Accounting Manager,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium,(071) 23 67 22 20,(071) 23 67 22 21,10252,SUPRD,4,2012-07-09,2012-08-06,2012-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium
5,HANAR,Hanari Carnes,Mario Pontes,Accounting Manager,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil,(21) 555-0091,(21) 555-8765,10253,HANAR,3,2012-07-10,2012-07-24,2012-07-16,2,58.17,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
6,CHOPS,Chop-suey Chinese,Yang Wang,Owner,Hauptstr. 29,Bern,Western Europe,3012,Switzerland,0452-076545,None,10254,CHOPS,5,2012-07-11,2012-08-08,2012-07-23,2,22.98,Chop-suey Chinese,Hauptstr. 31,Bern,Western Europe,3012,Switzerland
7,RICSU,Richter Supermarkt,Michael Holz,Sales Manager,Grenzacherweg 237,Genève,Western Europe,1203,Switzerland,0897-034214,None,10255,RICSU,9,2012-07-12,2012-08-09,2012-07-15,3,148.33,Richter Supermarkt,Starenweg 5,Genève,Western Europe,1204,Switzerland
8,WELLI,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,South America,08737-363,Brazil,(14) 555-8122,None,10256,WELLI,3,2012-07-15,2012-08-12,2012-07-17,2,13.97,Wellington Importadora,"Rua do Mercado, 12",Resende,South America,08737-363,Brazil
9,HILAA,HILARION-Abastos,Carlos Hernández,Sales Representative,Carrera 22 con Ave. Carlos Soublette #8-35,San Cristóbal,South America,5022,Venezuela,(5) 555-1340,(5) 555-1948,10257,HILAA,4,2012-07-16,2012-08-13,2012-07-22,3,81.91,HILARION-Abastos,Carrera 22 con Ave. Carlos Soublette #8-35,San Cristóbal,South America,5022,Venezuela


### Is there a statistically significant difference between the average order processing time in North America vs. Western Europe? 

In [123]:
western_europe = orders_cities.loc[(orders_cities['ShipRegion'] == 'Western Europe') & (orders_cities['Region'] == 'Western Europe')]
western_europe_dates = western_europe.iloc[:,13:17]
western_europe_dates.OrderDate = pd.to_datetime(western_europe_dates.OrderDate)
western_europe_dates.ShippedDate = pd.to_datetime(western_europe_dates.ShippedDate)

western_europe_dates['ProcessingTime'] = western_europe_dates.ShippedDate - western_europe_dates.OrderDate
processing_time_western_europe = western_europe_dates.ProcessingTime.dt.days

processing_time_western_europe = processing_time_western_europe.dropna()
processing_time_western_europe = processing_time_western_europe.values
processing_time_western_europe

array([12.,  5.,  7.,  2., 12.,  3.,  6., 10.,  8., 18.,  8.,  7., 10.,
        4.,  3.,  8.,  6.,  9.,  8.,  6.,  8., 29., 10., 10.,  7.,  5.,
        5.,  7.,  5., 10.,  5.,  6.,  7.,  8., 22.,  9., 12.,  9.,  7.,
       10., 11.,  3.,  8.,  3., 24., 21.,  3.,  3.,  8.,  8., 10.,  8.,
        6., 23.,  6.,  2.,  7., 10., 21., 35.,  4.,  6.,  8.,  7.,  5.,
        9., 20., 21.,  4.,  3.,  6.,  6.,  1.,  2.,  5.,  3., 21.,  8.,
        4.,  6., 12.,  8.,  3.,  8.,  7., 17., 27., 12.,  3.,  6., 24.,
       30.,  3.,  6., 21.,  2.,  2.,  4.,  2., 23.,  5., 25.,  6.,  4.,
        7.,  3.,  6.,  3.,  8.,  3.,  6., 17., 10.,  5., 17.,  4.,  7.,
        8., 35.,  4.,  7.,  9., 12.,  3.,  5.,  8.,  6.,  1.,  5.,  3.,
        6.,  7.,  8.,  1., 10., 17.,  3., 23., 10.,  7.,  8.,  2.,  7.,
        4.,  7.,  4.,  8., 19., 10.,  3.,  8.,  4.,  8.,  6.,  5.,  2.,
        2.,  9.,  8.,  1.,  7.,  6.,  5.,  9.,  2.,  7.,  9.,  2.,  6.,
        9.,  5.,  5.,  5.,  4., 10., 23.,  9.,  5.,  3., 29.,  7

In [122]:
north_america = orders_cities.loc[(orders_cities['ShipRegion'] == 'North America') & (orders_cities['Region'] == 'North America')]
north_america_dates = north_america.iloc[:,13:17]

north_america_dates
north_america_dates.OrderDate = pd.to_datetime(north_america_dates.OrderDate)
north_america_dates.ShippedDate = pd.to_datetime(north_america_dates.ShippedDate)

north_america_dates['ProcessingTime'] = north_america_dates.ShippedDate - north_america_dates.OrderDate
processing_time_north_america = north_america_dates.ProcessingTime.dt.days

processing_time_north_america = processing_time_north_america.dropna()
processing_time_north_america = processing_time_north_america.values
processing_time_north_america

array([ 3.,  9., 29.,  4.,  6., 26.,  8.,  7.,  9., 11., 10.,  2.,  8.,
        4.,  4.,  7.,  4.,  3.,  7.,  7.,  3.,  4.,  6.,  4.,  9.,  9.,
       10.,  9.,  5., 11.,  9.,  4.,  8.,  7.,  3., 18., 32.,  6.,  4.,
        2., 20., 32., 10.,  8.,  7.,  7., 10.,  3.,  9., 35.,  2.,  6.,
        7., 25.,  2., 11.,  7.,  9., 10.,  7.,  7., 32.,  4.,  5., 21.,
        8.,  3.,  4.,  5.,  4.,  7.,  3.,  9., 12., 10.,  6., 11., 37.,
        9.,  6., 23.,  2.,  5.,  4.,  6.,  6.,  5., 19.,  8.,  2.,  5.,
        9.,  6., 26.,  6., 11., 12.,  4.,  8.,  5., 18.,  6., 14., 10.,
        8.,  6.,  9., 29.,  6.,  7.,  8., 19.,  4.,  8., 18.,  8.,  9.,
        8.,  1.,  7.,  2.,  3.,  9.,  9.,  9.,  1.,  4., 10.,  9.,  2.,
       12., 10.,  4., 10.,  2.,  8., 10.,  2.,  8.,  3.,  4., 10.,  7.,
        6.,  7.,  6.,  3.,  3.])

In [125]:
def variance(sample):
    sample_mean = np.mean(sample)
    return sum([(i - sample_mean)**2 for i in sample])

def sample_variance(sample1, sample2):
    n_1, n_2 = len(sample1), len(sample2)
    var_1, var_2 = variance(sample1), variance(sample2)
    return (var_1 + var_2)/((n_1 + n_2)-2)

def twosample_tstatistic(disc, non_disc):
    disc_mean, non_disc_mean = np.mean(disc), np.mean(non_disc)
    samp_var = sample_variance(disc, non_disc)
    n_d, n_n = len(disc), len(non_disc)
    num = disc_mean - non_disc_mean
    denom = np.sqrt(samp_var * ((1/n_d)+(1/n_n)))
    return num / denom

def get_sample_sizes(disc, non_disc):
    n_1, n_2 = len(disc), len(non_disc)
    print(n_1, n_2)
    
def get_null_hypothesis_outcome(alpha,p):
    if p < alpha:
        return 'reject null hypothesis'
    else:
        return 'fail to reject null hypothesis'
    
def pass_terms_to_dict(sample_size, disc_rate, t_stat, p, outcome):
    return {'sample size':sample_size, 'discount rate': disc_rate, 't_stat': t_stat, 'p': p, 'outcome': outcome}

In [129]:
t_stat = twosample_tstatistic(processing_time_western_europe, processing_time_north_america)
t_stat

p = stats.t.sf(abs(t_stat), len(processing_time_western_europe)+len(processing_time_north_america)-1) * 2

alpha = 0.05

test_outcome = get_null_hypothesis_outcome(alpha,p)
test_outcome

'fail to reject null hypothesis'